
automatically blog about code changes over a specified time range and git database

blog about code?


In [ ]:
const path = require('path')
const fs = require('fs')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
const selectModel = importer.import('select llm')
const {spawnSync} = require('child_process')
const parsePatch = importer.import('parse patch file')
const {safeurl} = importer.import('domain cache tools')

const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'code-summaries')


async function blogAboutCode(project, timeframe = 3, promptModel = 'Meta') {

  if(typeof promptModel != 'function') {
    promptModel = await selectModel(promptModel)
  }

  // TODO: local pull
  if(!fs.existsSync(project)
    && fs.existsSync(path.join(PROFILE_PATH, project))) {
      project = path.join(PROFILE_PATH, project)
  }

  if(project.includes('://')
    && fs.existsSync(path.basename(project).replace('.git', ''))
  ) {
    project = path.join(PROFILE_PATH, path.basename(project).replace('.git', ''))
  }

  if(!fs.existsSync(project)) {
    console.error('No project')
    return
  }


  let result = await spawnSync('git', ['log', '--since=' + timeframe + '.days'], {
    cwd: project,
    timeout: 3000,
    stdio: ['pipe', 'pipe', 'pipe']
  })

  let commits = result.stdout.toString()

  //console.log(commits)
  const matches = [...commits.matchAll(/commit ([a-z0-9]+)/gi)]
  let first = matches[0]
  let last = matches[matches.length - 1]

  if(!first || !last) {
    console.error('No commits!')
    return
  }

  console.log(first[1], last[1])

  if(first == last) {
    result = await spawnSync('git', ['diff', '-U5', first[1]], {
      cwd: project,
      timeout: 3000,
      stdio: ['pipe', 'pipe', 'pipe']
    })
  } else {
    result = await spawnSync('git', ['diff', '-U5', last[1], first[1]], {
      cwd: project,
      timeout: 3000,
      stdio: ['pipe', 'pipe', 'pipe']
    })
  }

  let codePatch = result.stdout.toString()

  summary = parsePatch(codePatch)

  summaryOutputs = []
  let files = Object.keys(summary.files)
  for(let i = 0; i < files.length; i++) {
    if(files[i].match('cache.ipynb')) {
      continue
    }
    let q1 = 'Summerize this code like you\'re writing for a fun coding blog, include code excerpts as context:\n'
      + files[i] + '\nAdditions that were made:\n' + summary.files[files[i]].additions.join('')
      + '\nDeletions that were made:\n' + summary.files[files[i]].deletions.join('')
    console.log('User: ' + q1)
    let a1 = await promptModel(q1)
    console.log('AI: ' + a1)
    summaryOutputs.push(a1)
  }

  const mdHtml = md.render(summaryOutputs.join('\n'))
  let now = new Date()
  const filename = path.join(PROJECT_PATH, safeurl(path.basename(project) + now.getFullYear() + '-' + (now.getMonth() + 1) + '-' + now.getDate()) + '.html')
  fs.writeFileSync(filename, mdHtml)
  return mdHtml
}


module.exports = blogAboutCode



parse patch file?


In [ ]:
const fs = require("fs");

function parsePatch(patchFile) {
    let patchContent = patchFile
    if(fs.existsSync(patchFile)) {
        patchContent = fs.readFileSync(patchFile, "utf-8");
    }
    const lines = patchContent.split("\n");

    let summary = {
        files: {},
        totalAdditions: 0,
        totalDeletions: 0
    };

    let currentFile = null;

    lines.forEach(line => {
        console.log(line)
        if (line.startsWith("+++ b/")) {
            currentFile = line.replace("+++ b/", "").trim();
            summary.files[currentFile] = { additions: [], deletions: [] };
        } else if (line.startsWith("+") && !line.startsWith("+++")) {
            const addedLine = line.slice(1).trim(); // Remove leading '+' and trim
            summary.files[currentFile].additions.push(addedLine);
            summary.totalAdditions++;
        } else if (line.startsWith("-") && !line.startsWith("---")) {
            const deletedLine = line.slice(1).trim(); // Remove leading '+' and trim
            summary.files[currentFile].deletions.push(deletedLine);
            summary.totalDeletions++;
        } else if (line.startsWith('@')) {

        } else if (line.startsWith('diff')) {

        } else if (line.startsWith('index')) {

        } else if (!line.startsWith("---")) {
            const addedLine = line.slice(1).trim(); // Remove leading '+' and trim
            summary.files[currentFile].additions.push(addedLine);
        }
    });

    return summary;
}

module.exports = parsePatch




TODO: summerize file changes, downloads try to infer purpose, use file watcher? install time machine? how to i version my entire disk?
